In [18]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [19]:
from dataclasses import dataclass 
from langchain.tools import tool,ToolRuntime 

@tool 
def get_weather_for_location(city:str) -> str: 
    """ Get weather for a given city.""" 
    return f"It's always sunny in {city}!" 

@dataclass 
class Context: 
    """Custom runtime context schema.""" 
    user_id:str 

@tool 
def get_user_location(runtime: ToolRuntime[Context]) -> str: 
    """Retrieve user information based on User ID.""" 
    user_id = runtime.context.user_id 
    return 'Florida' if user_id =='1' else 'SF' 

In [25]:
import getpass 
import os 

if 'GOOGLE_API_KEY' not in os.environ: 
    os.environ['GOOGLE_API_KEY']=getpass.getpass('Enter your Google Gemini Api key') 

Enter your Google Gemini Api key ········


In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI 
# help(ChatGoogleGenerativeAI)  

In [53]:
model = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash',
                               temperature=1.0, 
                               max_tokens = None,
                               timeout= None,
                               max_retries=2,)

In [54]:
from langchain.chat_models import init_chat_model
# help(init_chat_model)

In [55]:
@dataclass
class ResponseFormat: 
    """ Response schema for the agent.""" 
    # Punny response (always required) 
    punny_response: str 
    # Any interesting information about the weather if available 
    weather_conditions: str | None=None 

In [56]:
from langgraph.checkpoint.memory import InMemorySaver 
checkpointer = InMemorySaver() 

In [59]:
from langchain.agents.structured_output import ToolStrategy 
from langchain.agents import create_agent 

agent = create_agent(model=model, 
                     system_prompt=SYSTEM_PROMPT, 
                     tools=[get_user_location,get_weather_for_location,],
                     context_schema=Context,
                     response_format=ToolStrategy(ResponseFormat), 
                     checkpointer=checkpointer 
                    ) 

config = {'configurable':{'thread_id':'1'}} 

response = agent.invoke( 
    {'messages':[{'role':'user','content':'what is the weather outside?'}]},
    config=config, 
    context=Context(user_id='1') 
) 

print(response['structured_response'])

import time 
time.sleep(60)
# Note that we can continue the conversation using the same `thread_id`.
config2 = {'configurable':{'thread_id':'2'}} 
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config2,
    context=Context(user_id="1")
)

print(response['structured_response'])
print(response)

ResponseFormat(punny_response="Looks like Florida is having a sun-sational day! Don't forget your shades, it's always bright there!", weather_conditions="It's always sunny in Florida!")
ResponseFormat(punny_response="You're most welcome! I'm just here to make sure your day isn't too 'cloudy' with confusion. Let me know if you need a 'forecast' of fun!", weather_conditions=None)
{'messages': [HumanMessage(content='thank you!', additional_kwargs={}, response_metadata={}, id='839bad97-a38b-4ba8-b3b0-0ba1632f455d'), HumanMessage(content='thank you!', additional_kwargs={}, response_metadata={}, id='f43dff1a-34ba-49e5-9a88-9ca32916dc1e'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'ResponseFormat', 'arguments': '{"punny_response": "You\'re most welcome! I\'m just here to make sure your day isn\'t too \'cloudy\' with confusion. Let me know if you need a \'forecast\' of fun!"}'}, '__gemini_function_call_thought_signatures__': {'97a5b5c5-5f4c-466a-b8f3-05bae9ff9b33': 'Cu